The single purpose of this notebook is to generate the mock data for the MCMC

In [ ]:
# general imports
import numpy    as np
from time      import time
from class_interface import Main

### Select input options and parameters


In [2]:
# These are the specifications of the experiment
specs = {'fsky': 0.35,
         'gal_per_arcmin': 30.,
         'sigma_eps': 0.3,
         'Nbin_ell': 30,
         'lmin': 10,
         'lmax': 2000,
         'zmin': 0.01,
         'kmax': 10, 
         'use_obs': {'GC': True, 'WL': True},
         'feedback': True, # We want to print stuff
         'ddm':False # We don't want ddm corrections on the non-linear Pk
        }

# path for n(z) and luminosity files
survey = {'luminosity': './input_files/luminosity_ratio.dat', 
          'nz': './input_files/nzTabISTF.dat',
          }


# fiducial values for cosmological parameters
params = {'output':'mPk',
          'non linear':'halofit',
          'omega_b': 0.022445,
          'omega_cdm': 0.1206,
          'n_s': 0.96,
          'ln10^{10}A_s': 3.0568,
          'H0': 67.,
          'tau_reio': 0.05,
          'N_ur':2.0328,
          'N_ncdm':1,
          'm_ncdm': 0.06
          }


if specs['ddm']:
    params.update({'log10_Gamma':-3,'log10_vk':2})
    
#Update with list of nuisance parameters
params.update({'A_IA': 1.72,'eta_IA': -0.41,'beta_IA': 0.0,
                   'b_1':1.09977,'b_2':1.22025,'b_3':1.2724,'b_4':1.31662,'b_5':1.35812,
                   'b_6':1.39982,'b_7':1.44465,'b_8':1.4965,'b_9':1.56525,'b_10':1.74299})


# path to save a mock dataset to. If an empty string is used, data will not be saved
if specs['ddm']:
    save_data_topath = '/data/StageIV_3x2pt_fid_LDDM.npy'
else:
    save_data_topath = '/data/StageIV_3x2pt_fid_LCDM.npy'

### Compute fiducial spectra

In [3]:
%%time 
calculator = Main(specs,survey)
observables = calculator.get_calculations(params)
Cls = observables.Cls
covmat_fiducial,noisy_cls = calculator.get_noisy_Cls(Cls)

CLASS calculation done in 2.3 s
Kernels computed in 0.2 s
Cls computed in 14.7 s
Covmat computed in 4.9 s
noisy realization around fiducial C_ell computed in 0.3 s


### Save the data

In [4]:
if save_data_topath != '':
    dataset = {'covmat': covmat_fiducial,
               'Cls': noisy_cls,
               'specs': specs,
               'nz': calculator.ni,
               'lumfunc': calculator.lumfunc}

    np.save(save_data_topath,dataset)